In [0]:
# Airflow Integration Parameters

try:
    # Create widgets for Airflow parameters
    dbutils.widgets.text("batch_id", "manual_run", "Batch ID from Airflow")
    dbutils.widgets.text("execution_date", "", "Execution Date from Airflow") 
    dbutils.widgets.text("force_refresh", "false", "Force data refresh")
    dbutils.widgets.text("quality_threshold", "0.8", "Data quality threshold")
    dbutils.widgets.text("dag_run_id", "", "DAG Run ID")
    
    # Get parameter values
    batch_id = dbutils.widgets.get("batch_id")
    execution_date = dbutils.widgets.get("execution_date")
    force_refresh = dbutils.widgets.get("force_refresh").lower() == "true"
    quality_threshold = float(dbutils.widgets.get("quality_threshold"))
    dag_run_id = dbutils.widgets.get("dag_run_id")
    
    print(f"🎯 Airflow Parameters:")
    print(f"   Batch ID: {batch_id}")
    print(f"   Execution Date: {execution_date}")
    print(f"   Force Refresh: {force_refresh}")
    print(f"   Quality Threshold: {quality_threshold}")
    print(f"   DAG Run ID: {dag_run_id}")
    
except Exception as e:
    print(f"⚠️ Widget creation failed (normal in some contexts): {e}")
    # Fallback values for manual runs
    batch_id = "manual_run"
    execution_date = ""
    force_refresh = False
    quality_threshold = 0.8
    dag_run_id = ""

In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # Event Hub Producer

# COMMAND ----------

# Install required libraries
%pip install newsapi-python requests azure-eventhub websocket-client

# COMMAND ----------

# Import libraries and handle SSL compatibility
import requests
import json
import asyncio
from datetime import datetime, timedelta, timezone
from azure.eventhub import EventHubProducerClient, EventData, TransportType
from azure.eventhub.exceptions import EventHubError
from pyspark.sql.functions import *
from pyspark.sql.types import *
from newsapi import NewsApiClient
import time

# Fix SSL compatibility issue for Event Hub in Databricks
try:
    import ssl
    if not hasattr(ssl, 'wrap_socket'):
        print("🔧 Applying SSL compatibility fix...")
        original_wrap_socket = ssl.wrap_socket
        def wrap_socket_fix(sock, **kwargs):
            context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
            return context.wrap_socket(sock, **kwargs)
        ssl.wrap_socket = wrap_socket_fix
        print("✅ SSL compatibility fix applied")
except Exception as e:
    print(f"⚠️ SSL fix warning: {e}")

print("✅ Libraries imported successfully")

# COMMAND ----------

# Configuration - Get credentials from Key Vault
try:
    polygon_api_key = dbutils.secrets.get(scope="stock-project", key="polygon-api-key")
    newsapi_key = dbutils.secrets.get(scope="stock-project", key="newsapi-key")
    
    eh_connection_string = dbutils.secrets.get(scope="stock-project", key="event-hub-connection-string")
    
    # Debug: Print connection string format (safely)
    print(f"🔍 Connection string format: {eh_connection_string[:50]}...")
    
    # Check if EntityPath is in connection string, if not add it
    if "EntityPath=" not in eh_connection_string:
        event_hub_name = "stock-data-hub"
        eh_connection_string = f"{eh_connection_string};EntityPath={event_hub_name}"
        print(f"✅ Added EntityPath: {event_hub_name}")
    else:
        entity_path_part = [part for part in eh_connection_string.split(';') if part.startswith('EntityPath=')]
        if entity_path_part:
            event_hub_name = entity_path_part[0].split('=')[1]
            print(f"✅ Found Event Hub name in connection string: {event_hub_name}")
    
    storage_account_key = dbutils.secrets.get(scope="stock-project", key="storage-account-key")
    print("✅ All credentials retrieved from Key Vault")
    print(f"✅ Event Hub connection string configured with EntityPath: {event_hub_name}")
except Exception as e:
    print(f"❌ Error retrieving secrets: {e}")
    import traceback
    traceback.print_exc()

# Storage configuration
storage_account_name = "dlsstocksentiment2025"
container_name = "data"
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)

adls_base_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net"
bronze_stock_path = f"{adls_base_path}/bronze/stock_data"
bronze_news_path = f"{adls_base_path}/bronze/news_data"

print("✅ Configuration complete")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Schema and Database Creation

# COMMAND ----------

def create_schemas_and_tables():
    """Create catalog, schema, and tables"""
    
    try:
        print("🏗️ Setting up database schema with ALL consistent data types...")
        
        # Get current catalog name
        current_catalog = spark.sql("SELECT current_catalog()").collect()[0][0]
        print(f"📁 Current catalog: {current_catalog}")
        
        # Create bronze schema if it doesn't exist
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {current_catalog}.bronze")
        print(f"✅ Schema {current_catalog}.bronze created/verified")
        
        # Create COMPLETELY NEW TABLES with unique names to avoid ALL conflicts
        new_stock_table = f"{current_catalog}.bronze.stock_data"
        new_news_table = f"{current_catalog}.bronze.news_data"
        
        # Drop any existing tables
        try:
            spark.sql(f"DROP TABLE IF EXISTS {new_stock_table}")
            spark.sql(f"DROP TABLE IF EXISTS {new_news_table}")
            print("🗑️ Dropped any existing tables")
        except:
            pass
        
        # Create stock_data table with ALL STRING types to eliminate conflicts
        stock_table_ddl = f"""
        CREATE TABLE {new_stock_table} (
            symbol STRING,
            timestamp STRING,
            open_price STRING,  -- STRING to avoid precision conflicts
            high_price STRING,  -- STRING to avoid precision conflicts
            low_price STRING,   -- STRING to avoid precision conflicts
            close_price STRING, -- STRING to avoid precision conflicts
            volume STRING,      -- STRING to avoid type conflicts
            source STRING,
            data_type STRING,
            market_cap_estimate STRING,  -- STRING to avoid precision conflicts
            daily_range STRING,          -- STRING to avoid precision conflicts
            api_fetch_time STRING,
            ingestion_time STRING,
            processed_date STRING,
            ingestion_batch STRING,
            ingestion_source STRING,
            stream_attempted STRING      -- STRING instead of BOOLEAN
        )
        USING DELTA
        """
        
        spark.sql(stock_table_ddl)
        print(f"✅ Table {new_stock_table} created with ALL STRING types")
        
        # Create news_data table with ALL STRING types to eliminate conflicts
        news_table_ddl = f"""
        CREATE TABLE {new_news_table} (
            title STRING,
            content STRING,
            source STRING,
            published_at STRING,
            url STRING,
            sentiment_score STRING,     -- STRING to avoid precision conflicts
            data_type STRING,
            title_length STRING,        -- STRING to avoid INT/LONG conflicts
            content_length STRING,      -- STRING to avoid INT/LONG conflicts
            api_fetch_time STRING,
            sentiment_category STRING,
            ingestion_time STRING,
            processed_date STRING,
            ingestion_batch STRING,
            ingestion_source STRING,
            stream_attempted STRING     -- STRING instead of BOOLEAN
        )
        USING DELTA
        """
        
        spark.sql(news_table_ddl)
        print(f"✅ Table {new_news_table} created with ALL STRING types")
        
        # Verify tables exist
        tables = spark.sql(f"SHOW TABLES IN {current_catalog}.bronze").collect()
        table_names = [row.tableName for row in tables]
        print(f"📋 Available tables in bronze schema: {table_names}")
        
        return current_catalog, new_stock_table, new_news_table
        
    except Exception as e:
        print(f"❌ Error creating schemas: {e}")
        import traceback
        traceback.print_exc()
        return None, None, None

# Create schemas and get catalog name
current_catalog, stock_table_name, news_table_name = create_schemas_and_tables()

# COMMAND ----------

# MAGIC %md
# MAGIC ## Event Hub Producer Functions

# COMMAND ----------

class EventHubProducer:
    def __init__(self, connection_string):
        self.connection_string = connection_string
        self.client = None
    
    def connect(self):
        """Initialize Event Hub producer client"""
        connection_methods = [
            ("WebSocket", TransportType.AmqpOverWebsocket),
            ("Auto-detect", None),
            ("Standard AMQP", TransportType.Amqp)
        ]
        
        for method_name, transport_type in connection_methods:
            try:
                print(f"🔌 Trying {method_name} connection...")
                
                if transport_type:
                    self.client = EventHubProducerClient.from_connection_string(
                        self.connection_string,
                        transport_type=transport_type
                    )
                else:
                    self.client = EventHubProducerClient.from_connection_string(
                        self.connection_string
                    )
                
                # Test the connection by creating a batch
                test_batch = self.client.create_batch()
                
                print(f"✅ Connected to Event Hub using {method_name}")
                return True
                
            except Exception as e:
                print(f"❌ {method_name} failed: {str(e)[:60]}...")
                if self.client:
                    try:
                        self.client.close()
                    except:
                        pass
                    self.client = None
                continue
        
        print("❌ All connection methods failed")
        return False
    
    def send_batch_data(self, data_records, data_type):
        """Send batch of data to Event Hub"""
        if not self.client:
            print("❌ Event Hub client not connected")
            return False
        
        try:
            event_data_batch = self.client.create_batch()
            sent_count = 0
            
            for record in data_records:
                # Add event metadata
                enhanced_record = record.copy()
                enhanced_record.update({
                    'event_timestamp': datetime.now(timezone.utc).isoformat(),
                    'data_type': data_type,
                    'event_source': 'api_producer',
                    'partition_key': record.get('symbol', 'general'),
                    'processing_mode': 'streaming'
                })
                
                # Create event data
                event_data = EventData(json.dumps(enhanced_record))
                
                # Set partition key for better distribution
                if 'symbol' in record:
                    event_data.properties = {'partition_key': record['symbol']}
                
                try:
                    event_data_batch.add(event_data)
                    sent_count += 1
                except ValueError:
                    # Batch is full, send it and create a new one
                    self.client.send_batch(event_data_batch)
                    event_data_batch = self.client.create_batch()
                    event_data_batch.add(event_data)
                    sent_count += 1
            
            # Send remaining events
            if event_data_batch:
                self.client.send_batch(event_data_batch)
            
            print(f"✅ Sent {sent_count} {data_type} events to Event Hub")
            return True
            
        except EventHubError as e:
            print(f"❌ Event Hub error: {e}")
            return False
        except Exception as e:
            print(f"❌ Unexpected error sending to Event Hub: {e}")
            return False
    
    def close(self):
        """Close Event Hub connection"""
        if self.client:
            self.client.close()
            print("✅ Event Hub connection closed")

# Initialize Event Hub producer
eh_producer = EventHubProducer(eh_connection_string)

# COMMAND ----------

# MAGIC %md
# MAGIC ## Enhanced API Data Fetching (ALL DATA TYPES AS STRINGS)

# COMMAND ----------

def get_stock_data_all_strings(symbols):
    """Fetch stock data with ALL data as strings to avoid type conflicts"""
    stock_data = []
    
    print(f"🔄 Fetching stock data for {len(symbols)} symbols...")
    
    for symbol in symbols:
        try:
            url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/prev"
            params = {"apikey": polygon_api_key}
            
            response = requests.get(url, params=params)
            response.raise_for_status()
            
            data = response.json()
            
            if 'results' in data and len(data['results']) > 0:
                result = data['results'][0]
                
                # Convert timestamp to string consistently
                trade_timestamp = datetime.fromtimestamp(result['t'] / 1000)
                
                # ALL VALUES AS STRINGS to eliminate type conflicts
                stock_record = {
                    "symbol": str(symbol),
                    "timestamp": str(trade_timestamp.isoformat()),
                    "open_price": str(result['o']),      # STRING
                    "high_price": str(result['h']),     # STRING
                    "low_price": str(result['l']),      # STRING
                    "close_price": str(result['c']),    # STRING
                    "volume": str(result['v']),         # STRING
                    "source": str("polygon.io"),
                    "data_type": str("stock_price"),
                    "market_cap_estimate": str(float(result['c']) * int(result['v'])),  # STRING
                    "daily_range": str(float(result['h']) - float(result['l'])),       # STRING
                    "api_fetch_time": str(datetime.now(timezone.utc).isoformat())
                }
                
                stock_data.append(stock_record)
                print(f"✅ Retrieved {symbol}: ${result['c']} (Volume: {result['v']:,})")
            else:
                print(f"⚠️ No data available for {symbol}")
                
        except Exception as e:
            print(f"❌ Error fetching {symbol}: {e}")
        
        # Rate limiting
        if symbol != symbols[-1]:
            print("⏳ Waiting 12 seconds for rate limiting...")
            time.sleep(12)
    
    return stock_data

def get_news_data_all_strings(query="stock market", page_size=10):
    """Fetch news data with ALL data as strings to avoid type conflicts"""
    try:
        print(f"🔄 Fetching news articles for query: '{query}'")
        
        newsapi = NewsApiClient(api_key=newsapi_key)
        
        from_date = datetime.now() - timedelta(days=1)
        
        articles = newsapi.get_everything(
            q=query,
            from_param=from_date.strftime('%Y-%m-%d'),
            language='en',
            sort_by='publishedAt',
            page_size=page_size
        )
        
        news_data = []
        
        for article in articles['articles']:
            if article['title'] and (article['content'] or article['description']):
                content = article['content'] or article['description'] or ""
                
                # Enhanced sentiment scoring
                positive_words = ['surge', 'rise', 'gain', 'profit', 'growth', 'bullish', 'positive', 'up', 'increase', 'strong']
                negative_words = ['fall', 'drop', 'loss', 'decline', 'bearish', 'negative', 'crash', 'down', 'decrease', 'weak']
                
                title_lower = article['title'].lower()
                content_lower = content.lower()
                
                sentiment_score = 0.0
                for word in positive_words:
                    if word in title_lower or word in content_lower:
                        sentiment_score += 0.1
                
                for word in negative_words:
                    if word in title_lower or word in content_lower:
                        sentiment_score -= 0.1
                
                # Clamp sentiment score
                if sentiment_score > 1.0:
                    sentiment_score = 1.0
                elif sentiment_score < -1.0:
                    sentiment_score = -1.0
                
                # Round sentiment score
                sentiment_score_rounded = float(int(sentiment_score * 100) / 100)
                
                # Determine sentiment category
                if sentiment_score_rounded > 0.1:
                    sentiment_cat = "positive"
                elif sentiment_score_rounded < -0.1:
                    sentiment_cat = "negative"
                else:
                    sentiment_cat = "neutral"
                
                # ALL VALUES AS STRINGS to eliminate type conflicts
                news_record = {
                    "title": str(article['title'][:200]) if article['title'] else "",
                    "content": str(content[:500]),
                    "source": str(article['source']['name']) if article['source']['name'] else "",
                    "published_at": str(article['publishedAt']) if article['publishedAt'] else "",
                    "url": str(article['url']) if article['url'] else "",
                    "sentiment_score": str(sentiment_score_rounded),    # STRING
                    "data_type": str("news_sentiment"),
                    "title_length": str(len(article['title'])) if article['title'] else "0",     # STRING
                    "content_length": str(len(content)),                                         # STRING
                    "api_fetch_time": str(datetime.now(timezone.utc).isoformat()),
                    "sentiment_category": str(sentiment_cat)
                }
                
                news_data.append(news_record)
        
        print(f"✅ Retrieved {len(news_data)} news articles")
        return news_data
        
    except Exception as e:
        print(f"❌ Error fetching news: {e}")
        import traceback
        traceback.print_exc()
        return []

# COMMAND ----------

# MAGIC %md
# MAGIC ## Hybrid Data Pipeline

# COMMAND ----------

def run_data_pipeline():
    """Main pipeline: API → Event Hubs → Bronze Backup"""
    
    print("🚀 Starting Data Pipeline...")
    print("📊 Phase 1: API Data Collection")
    
    # Define stock symbols
    stock_symbols = ["AAPL", "GOOGL", "MSFT", "AMZN", "META", "TSLA"]
    
    # Fetch data from APIs
    print(f"📈 Fetching stock data for: {', '.join(stock_symbols)}")
    stock_data = get_stock_data_all_strings(stock_symbols)
    
    print(f"📰 Fetching financial news...")
    news_data = get_news_data_all_strings("stock market financial", 8)
    
    print(f"\n📊 Data Collection Summary:")
    print(f"   Stock records: {len(stock_data)}")
    print(f"   News records: {len(news_data)}")
    
    # Phase 2: Stream to Event Hubs
    print(f"\n📡 Phase 2: Streaming to Event Hubs")
    
    if eh_producer.connect():
        streaming_success = True
        
        # Send stock data
        if stock_data:
            print(f"📈 Streaming {len(stock_data)} stock records...")
            if not eh_producer.send_batch_data(stock_data, "stock_price"):
                streaming_success = False
        
        # Send news data  
        if news_data:
            print(f"📰 Streaming {len(news_data)} news records...")
            if not eh_producer.send_batch_data(news_data, "news_sentiment"):
                streaming_success = False
        
        eh_producer.close()
        
        if streaming_success:
            print("✅ Successfully streamed all data to Event Hubs")
        else:
            print("⚠️ Some data failed to stream - proceeding with backup")
    else:
        print("⚠️ Event Hub connection failed - proceeding with direct Bronze save")
        streaming_success = False
    
    # Phase 3: Backup to Bronze Layer
    print(f"\n💾 Phase 3: Backup to Bronze Layer")
    save_to_bronze_all_strings(stock_data, news_data)
    
    return len(stock_data), len(news_data)

def save_to_bronze_all_strings(stock_data, news_data):
    """Save data directly to Bronze layer"""
    
    if not current_catalog:
        print("❌ Schema not available - cannot save to Bronze")
        return
    
    batch_id = f"batch_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    current_time_str = datetime.now(timezone.utc).isoformat()
    current_date_str = datetime.now().strftime('%Y-%m-%d')
    
    # Save stock data
    if stock_data:
        try:
            # Add metadata as strings
            for record in stock_data:
                record["ingestion_time"] = str(current_time_str)
                record["processed_date"] = str(current_date_str)
                record["ingestion_batch"] = str(batch_id)
                record["ingestion_source"] = str("producer")
                record["stream_attempted"] = str(True)  # STRING instead of boolean
            
            stock_df = spark.createDataFrame(stock_data)
            
            # Save to Unity Catalog (managed tables) - NO SCHEMA MERGING
            (stock_df.write
             .format("delta")
             .mode("append")
             .option("mergeSchema", "false")
             .saveAsTable(stock_table_name))
            
            print(f"✅ Saved {len(stock_data)} stock records to Unity Catalog")
            
            # Try to backup to ADLS2
            try:
                (stock_df.write
                 .format("delta")
                 .mode("append")
                 .option("mergeSchema", "false")
                 .save(bronze_stock_path))
                print(f"✅ Also backed up stock data to ADLS2")
            except Exception as adls_error:
                print(f"⚠️ ADLS2 backup failed: {str(adls_error)[:100]}...")
            
        except Exception as e:
            print(f"❌ Error saving stock backup: {e}")
            import traceback
            traceback.print_exc()
    
    # Save news data
    if news_data:
        try:
            # Add metadata as strings
            for record in news_data:
                record["ingestion_time"] = str(current_time_str)
                record["processed_date"] = str(current_date_str)
                record["ingestion_batch"] = str(batch_id)
                record["ingestion_source"] = str("producer")
                record["stream_attempted"] = str(True)  # STRING instead of boolean
            
            news_df = spark.createDataFrame(news_data)
            
            # Save to Unity Catalog (managed tables) - NO SCHEMA MERGING
            (news_df.write
             .format("delta")
             .mode("append")
             .option("mergeSchema", "false")
             .saveAsTable(news_table_name))
            
            print(f"✅ Saved {len(news_data)} news records to Unity Catalog")
            
            # Try to backup to ADLS2
            try:
                (news_df.write
                 .format("delta")
                 .mode("append")
                 .option("mergeSchema", "false")
                 .save(bronze_news_path))
                print(f"✅ Also backed up news data to ADLS2")
            except Exception as adls_error:
                print(f"⚠️ ADLS2 backup failed: {str(adls_error)[:100]}...")
            
        except Exception as e:
            print(f"❌ Error saving news backup: {e}")
            import traceback
            traceback.print_exc()

# COMMAND ----------

# MAGIC %md
# MAGIC ## Execute Pipeline

# COMMAND ----------

# Run the pipeline
try:
    stock_count, news_count = run_data_pipeline()
    
    print(f"\n🎉 Pipeline Complete!")
    print(f"✅ Processed: {stock_count} stocks, {news_count} news articles")
    print(f"📡 Data streamed to Event Hubs for real-time processing")
    print(f"💾 Data backed up to Bronze layer for reliability")
    
    # Verify bronze layer
    print(f"\n🔍 Bronze Layer Verification:")
    
    try:
        if stock_table_name and news_table_name:
            total_stock = spark.table(stock_table_name).count()
            total_news = spark.table(news_table_name).count()
            
            print(f"   Total stock records: {total_stock}")
            print(f"   Total news records: {total_news}")
            
            # Show recent data
            if total_stock > 0:
                print(f"\n📊 Recent Stock Data:")
                (spark.table(stock_table_name)
                 .select("symbol", "close_price", "volume", "ingestion_time", "ingestion_source")
                 .orderBy(col("ingestion_time").desc())
                 .limit(5)
                 .show())
            
            if total_news > 0:
                print(f"\n📰 Recent News Data:")
                (spark.table(news_table_name)
                 .select("title", "sentiment_score", "ingestion_time", "sentiment_category", "ingestion_source")
                 .orderBy(col("ingestion_time").desc())
                 .limit(3)
                 .show(truncate=False))
        
    except Exception as e:
        print(f"❌ Error in verification: {e}")
        import traceback
        traceback.print_exc()

except Exception as e:
    print(f"❌ Pipeline failed: {e}")
    import traceback
    traceback.print_exc()

# COMMAND ----------

print("🔄 Event Hub Producer Complete!")
print(f"📋 Stock Table: {stock_table_name}")
print(f"📋 News Table: {news_table_name}")
print(f"\n⏰ Execution completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
🔧 Applying SSL compatibility fix...
⚠️ SSL fix warning: module 'ssl' has no attribute 'wrap_socket'
✅ Libraries imported successfully
🔍 Connection string format: Endpoint=sb://eh-stock-sentiment-2025.servicebus.w...
✅ Added EntityPath: stock-data-hub
✅ All credentials retrieved from Key Vault
✅ Event Hub connection string configured with EntityPath: stock-data-hub
✅ Configuration complete
🏗️ Setting up database schema with ALL consistent data types...
📁 Current catalog: databricks_stock_sentiment_canada
✅ Schema databricks_stock_sentiment_canada.bronze created/verified
🗑️ Dropped any existing tables
✅ Table databricks_stock_sentiment_canada.bronze.stock_data created with ALL STRING types
✅ Table databricks_stock_sentiment_canada.bronze.news_data created with ALL STRING types
📋 Available tables in bronze schema: ['news_data', 'stock_data']
🚀 Starting Data Pipeline..

In [0]:
%python
import json
from datetime import datetime

# Airflow Integration - Success/Failure Reporting

try:
    # If we reach here, notebook executed successfully
    success_result = {
        "status": "SUCCESS",
        "message": "Notebook execution completed successfully",
        "batch_id": batch_id,
        "execution_timestamp": datetime.now().isoformat(),
        "records_processed": locals().get('total_records_processed', 0),  # Update based on your variables
        "data_quality_score": locals().get('data_quality_score', 1.0)     # Update based on your variables
    }
    
    print(f"✅ Notebook Success:")
    print(json.dumps(success_result, indent=2))
    
    # Exit with success status for Airflow
    dbutils.notebook.exit(success_result)
    
except Exception as e:
    # If any error occurs, report failure
    failure_result = {
        "status": "FAILED", 
        "message": f"Notebook execution failed: {str(e)}",
        "batch_id": batch_id,
        "execution_timestamp": datetime.now().isoformat(),
        "error_type": type(e).__name__
    }
    
    print(f"❌ Notebook Failure:")
    print(json.dumps(failure_result, indent=2))
    
    # Exit with failure status for Airflow
    dbutils.notebook.exit(failure_result)